# Lecture 4: Grover search algorithm

<center><img src="../figures/qibo_mascotte/qibo_coffee.png" alt="drawing" width="400"/></center>
<center><strong>Fig. 4:</strong> Who put sugar in Qibo's coffee? [DALL-E].</center>

## Introduction

The Grover algorithm is probably one of the most famous quantum computing proof of utility. 

It has been known as "Grover search algorithm" because it is effective in a context in which we aim to find a specific solution of a problem among a set of possible solutions. 

#### Problem setup

In this notebook, we are going to implement the Grover algorithm using a system of $N$ qubits in order to explore a set of $M=2^N$ possible solutions and, without going deep into the mathematical details, we will proof we need a number of Grover iterations proportional to $\sqrt{M}$ to find the target solution with probability close to one.

---

To give the lesson a more concrete aspect, let's take a more realistic problem: 
Suppose you're at a party with $M=2^N$ friends. While you're peacefully drinking your coffee, you get distracted for a moment, and someone, without you noticing, pours sugar into your coffee.

It's a disaster because you like your coffee bitter! So, you absolutely need to find the person responsible for this.

Let's also suppose that there is no concrete evidence that can lead us to the guilty person, however, while poursing coffee inside your mug, the thief left a hair inside the cup, which can be used to extract DNA.

Perfect! we can compare the guilty DNA with that of every person who was at the party at that moment.

---

This problem can be **classically** tackled by checking all the people one by one and, in the worst case, it takes $M = 2^N$ attempts before finding the guilty person in the group. On the other hand, using quantum computing and the Grover algorithm, we will see the solution can be found with a number of algorithm's calls that scales as $\sqrt{M}$: *we gain a quadratic speedup!*

Let's start investigating the culprit by importing some packages.

In [ ]:
# if you don't have already qibo, qibojit and qiboedu installed in your computer
# uncomment and execute the following lines 

# !pip install qibo
# !pip install qibojit
# !pip install git+https://github.com/qiboteam/qiboedu

In [ ]:
# import random, numpy, matplotlib

# import qibo and its modules

# import all utils and plotscripts modules from scripts

# set qibo backend

Secondly, we need to generate a set of elements which can deal with a quantum computer and are able to represent all the possible solutions we want to explore. In practice, we create a set of $2^N$ bitstrings and we associate every person at the party to one of the bistrings. 

We use the function `scripts.utils.generate_bitstring_combination` to generate a list containing all the possible bitstrings of zeros and ones composed of $N$ bits.

In [ ]:
# set the problem dimensionality --> the size of the set to be explored

# set the number of shots (circuit's executions)
# we do this because it's more realistic e.g. 100000

# generate bitstrings list

In [ ]:
# check whether the length of the list is equal to 2**n

Let's generate the list of people who were at the party.

In [ ]:
# !pip install Faker 

# import faker.Faker 

# build a faker object calling Faker()

In [ ]:
# generate 2**n fake names with faker.name()

In practice, we have now a list of strings, in which the element of index $k$ represents the $k$-th suspect among the people.

Let's assume the guilty guy in the list is the number 11!

In [ ]:
# set the guilty_index

# visualize some info

### Grover algorithm main ingredients

Now the problem is set up and we can focus on the Grover implementation. 

<center><img src="../figures/grover-circ.png" alt="drawing" width="800"/></center>
We need to follow some steps:

1. we prepare a system of $N+1$ qubits into the $|0\rangle^{\otimes N} |1\rangle$ state. The ancilla gate is the one prepared as usual into $|1\rangle$;
2. we apply Hadamard gates to all the qubits in order to create superposition of all the $2^N$ states of the input register. We call this superposed state $|s\rangle$;
3. we apply an **oracle** function $U_f$ to the qubits, which is constructed in order to detect the guilty state among the others. In our narrative, it can be seen as the machine which process the DNA of the people, allowing to find the guilty person. In a quantum computer notation, it is a set of operation which flips the sign of the amplitude of the state corresponding to the guilty guy (the number 11 in our case).
4. we apply the **diffusion operator** $U_s = 2|s\rangle \langle s| - I$, whose effect consists in reflecting the amplitudes of the system about the average amplitude. In practice, acting together with the oracle over the superposed states, it increases the amplitude of the guilty state and decrease the one of the others.
5. we repeat the process a number of time proportional to $\sqrt{M}$ to find the guilty guy state with probability close to one.

We see some more detail in the following cells. For now, we can start implementing all the elements of the Grover algorithm.

#### Initial and final layers of the circuit

Regardless of the number of times we repeat the application of $U_f$ and $U_s$, we need to build the initial and the final circuit layer.

In [ ]:
# The Grover circuit will be composed of many layers
# the first and the last layer are always the same and we call the initial_layer and final_layer

# write a function to set the circuit and initial superposition

# write a function to build a circuit containing the final measurements

Let's visualize the initial and final layer.

In [ ]:
# build the circuit as initial_layer + final_layer and draw it

With this setup, the final state is the equally superposed state and we expect uniformly distributed probabilities between all the available states.
To visualize it, we can use the `scripts.plotscripts.plot_grover_probabilities` function, which shows the probability of each state calculated using the collected frequencies after executing the circuit `nshots` times.

In [ ]:
# use plot_input_register_amplitudes passing the quantum state

In [ ]:
# use plot_grover_probabilities which requires the circuit and nshots
# here we are using the frequencies, and the result is no more exact

As we can see from the plot, the state is prepared into $|s\rangle$.

#### Oracle 

We need now to construct the oracle $U_f$. This can be done firstly building a function:

$$ f: \{0,1\}^n \to \{0,1\} $$

which returns $1$ in the only case in which the input variable $x$ is the one we are searching for, otherwise it returns $0$. Given this, the oracle $U_f$ acts on the $N$-qubits state $|x\rangle$ in the following way: $$ U_f |x\rangle = (-1)^{f(x)}|x\rangle, $$

flipping the amplitude of the state only if $|x\rangle$ is the guilty one:

$$ 
\begin{cases}
U_f\,|x\rangle = - |x\rangle \qquad \text{if} \quad |x\rangle \quad \text{guilty}, \\
U_f\,|x\rangle = + |x\rangle \qquad \text{if} \quad |x\rangle \quad \text{innocent}. \\
\end{cases}
$$

The easiest way to implement such an operator consists in building a multi-controlled gate which is "triggered" by the guilty state only. It means we need to define a gate which acts on the ancilla and, with its action, it provokes a phase kickback effect on the guilty state only. 

Since the multi-controlled gate is typically activated by the component $|1\rangle^{\otimes N}$ of the input register state, if the guilty state is the $|1\rangle^{\otimes N}$ itself, our job gets easier: in fact, the multi-controlled gate will already do the job we want to do. 

Otherwise, if the guilty state is something different from $|1\rangle^{\otimes N}$, we just need to manipulate the input register state so that the component which activates the gate is the guilty one instead. A simple way of doing this is apply a NOT gate to each qubit of the system which, in the bitstring, corresponds to a zero. 

In [ ]:
# define a function which implements the oracle given guilty_index

Let's have a look to the circuit.

In [ ]:
# build the oracle circuit and draw it

Execute the whole circuit involving the oracle.

In [ ]:
# combine together initial_layer + oracle + final_layer and draw the circuit

We can visualize the amplitudes of the input register using the function `scripts.plotscripts.plot_input_register_amplitudes`.

In [ ]:
# plot the input register amplitudes using plot_input_register_amplitudes

Even though the amplitude of the guilty state is flipped, if we look to the probabilies associated to the states of the input register we still collect an equally superposed state.

In [ ]:
# plot the probabilities associated to this state using the plot_grover_probabilities function
# it requires the circuit and nshots as arguments

#### The diffusion operator

The next step in Grover is to implement the diffusion operator, whose effect is to amplify the guilty amplitude while decreasing the others.
Typically it is implemented using a multi-controlled Z gate acting on the ancilla e controlled by the input register. This multi-controlled Z has to be preceded and followed by a layer of Hadamards applied to the imput register. For now, we'll ask you to accept this statement as it is. In the notebook corresponding to the Lecture 5 we delve deeper in the details of this implementation.

One way to practically implement the CZ consists in applying an X gate to all the qubits of the input register, then apply a multi-controlled NOT gate on the ancilla and controlled by the whole input register. Finally, we have to re-apply the X gates to the input register. 

In [ ]:
# implement the diffusion operator

In [ ]:
# function to apply nsteps of Grover

In [ ]:
# build grover circuit with nsteps=1

# draw the circuit

In [ ]:
# plot_grover_probabilities, passing grover circuit and nshots

In [ ]:
# build grover circuit with nsteps=2

# draw the circuit

In [ ]:
# plot_grover_probabilities, passing grover circuit and nshots

From the plot and the amplitudes of the final state it is clearer and clearer who is the guilty person!
In the following cell we check explicitly who is the guilty person.

In [ ]:
# collect frequencies with the current circuit

# write the corresponding probabilities dividing by nshots

# save index corresponding to max probability and show the corresponding name

We expect the optimal number of Grover steps is $\sim\pi/4 \sqrt{M}$ from the theoretical results.
Let's check what happens with a growing number of steps.

In [ ]:
# compute the optimal number of iterations

In [ ]:
# allocate empty max probabilities list

# set the number of experiments, e.g. for nsteps up to 10 and use build_grover(n, nsteps)

# check for number of steps in np.arange(0, nsteps) using the utils.compute_max_probability(c, nshots) function

Plot the sequence of registered probabilities.

In [ ]:
# plot the max_probability in terms of niterations

In [ ]:
# build grover circuit with optimal number of steps

# draw the circuit

In [ ]:
# use plotscripts.plot_grover_probabilities